In [4]:

#do not delete
import os
import sys
import json
import subprocess
import shutil
from pathlib import Path
from datetime import datetime

WORK_DIR = os.getcwd()
DATA_ROOT = "/courses/CS5330.202610/data/FinalProject"

print("=" * 80)
print("SURVUE - TRAINING PIPELINE")
print("=" * 80)

subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "ultralytics>=8.0.200", "--no-cache-dir"])

from ultralytics import YOLO
import torch
import yaml

print("\nSTEP 1: Dataset")
print("-" * 40)

if os.path.exists(f"{WORK_DIR}/dataset/data.yaml"):
    print("Dataset found")

    data_yaml_content = f"""path: {WORK_DIR}/dataset
train: images/train
val: images/val
nc: 5
names: ['class1', 'class2', 'class3', 'class4', 'class5']
hsv_h: 0.025
hsv_s: 0.8
hsv_v: 0.5
degrees: 20
translate: 0.25
scale: 0.9
shear: 15
perspective: 0.0003
flipud: 0.4
fliplr: 0.5
mosaic: 1.0
mixup: 0.3
copy_paste: 0.5
paste_in: 0.2
loss_ota: 1
"""

    with open(f"{WORK_DIR}/dataset/data.yaml", 'r') as f:
        existing_data = yaml.safe_load(f)
        nc = existing_data.get('nc', 5)
        names = existing_data.get('names', ['class1', 'class2', 'class3', 'class4', 'class5'])

    data_yaml_content = data_yaml_content.replace("nc: 5", f"nc: {nc}")
    data_yaml_content = data_yaml_content.replace(
        "['class1', 'class2', 'class3', 'class4', 'class5']",
        str(names)
    )

    with open(f"{WORK_DIR}/dataset/data_recall.yaml", 'w') as f:
        f.write(data_yaml_content)

    print("Created data_recall.yaml")
else:
    print("Dataset not found.")
    sys.exit(1)

print("\nSTEP 2: Training Settings")
print("-" * 40)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

training_configs = {
    "strategy_1_medium": {
        "name": "YOLOv8m_Recall",
        "model": "yolov8m.pt",
        "epochs": 100,
        "batch": 8 if device == 'cuda' else 4,
        "config": {
            "imgsz": 640,
            "patience": 30,
            "optimizer": "AdamW",
            "lr0": 0.0005,
            "lrf": 0.01,
            "momentum": 0.937,
            "weight_decay": 0.0003,
            "warmup_epochs": 5,
            "warmup_momentum": 0.8,
            "box": 4.0,
            "cls": 1.5,
            "dfl": 1.5,
            "label_smoothing": 0.1,
            "nbs": 64,
            "close_mosaic": 20,
            "conf": 0.001
        }
    },
    "strategy_2_large": {
        "name": "YOLOv8l_Recall",
        "model": "yolov8l.pt",
        "epochs": 80,
        "batch": 4 if device == 'cuda' else 2,
        "config": {
            "imgsz": 800,
            "patience": 25,
            "optimizer": "AdamW",
            "lr0": 0.0003,
            "lrf": 0.01,
            "momentum": 0.937,
            "weight_decay": 0.0002,
            "warmup_epochs": 7,
            "box": 3.0,
            "cls": 2.0,
            "dfl": 1.5,
            "label_smoothing": 0.15,
            "conf": 0.001
        }
    },
    "strategy_3_ensemble": {
        "name": "YOLOv8s_Recall",
        "model": "yolov8s.pt",
        "epochs": 120,
        "batch": 16 if device == 'cuda' else 8,
        "config": {
            "imgsz": 640,
            "patience": 30,
            "optimizer": "SGD",
            "lr0": 0.008,
            "lrf": 0.01,
            "momentum": 0.95,
            "weight_decay": 0.0005,
            "warmup_epochs": 5,
            "box": 5.0,
            "cls": 1.0,
            "dfl": 1.5,
            "conf": 0.01
        }
    }
}

def train_model(strategy_name, config):
    print(f"\nTraining {strategy_name}")
    print("-" * 40)

    model = YOLO(config["model"])

    train_args = {
        "data": f"{WORK_DIR}/dataset/data_recall.yaml",
        "epochs": config["epochs"],
        "batch": config["batch"],
        "device": device,
        "project": f"{WORK_DIR}/runs_recall",
        "name": config["name"],
        "exist_ok": True,
        "save": True,
        "save_period": 10,
        "verbose": True,
        "val": True,
        "plots": True,
        "workers": 4 if device == 'cuda' else 0,
        "mosaic": 1.0,
        "mixup": 0.3,
        "copy_paste": 0.4,
        "flipud": 0.3,
        "fliplr": 0.5,
        "erasing": 0.4,
        "crop_fraction": 1.0,
        "scale": 0.9,
        "translate": 0.2,
        "degrees": 15,
        "shear": 10,
        **config["config"]
    }

    try:
        model.train(**train_args)
        return True, f"{WORK_DIR}/runs_recall/{config['name']}/weights/best.pt"
    except Exception as e:
        print(f"Training error: {e}")
        return False, None

print("\nSTEP 3: Strategy Selection")
print("-" * 40)

if device == "cuda":
    if torch.cuda.get_device_properties(0).total_memory > 10e9:
        selected = "strategy_2_large"
    else:
        selected = "strategy_1_medium"
else:
    selected = "strategy_3_ensemble"

config = training_configs[selected]
success, model_path = train_model(selected, config)

print("\nSTEP 4: Validation")
print("-" * 40)

if success and model_path:
    model = YOLO(model_path)

    for conf in [0.001, 0.005, 0.01]:
        m = model.val(
            data=f"{WORK_DIR}/dataset/data.yaml",
            conf=conf,
            iou=0.3,
            max_det=3000,
            device=device,
            batch=4,
            verbose=False
        )

        p = float(m.box.mp) if hasattr(m.box, "mp") else 0
        r = float(m.box.mr) if hasattr(m.box, "mr") else 0
        m50 = float(m.box.map50) if hasattr(m.box, "map50") else 0
        m95 = float(m.box.map) if hasattr(m.box, "map") else 0
        f1 = (2 * p * r) / (p + r + 1e-6)

        print(f"conf={conf:.3f}: P={p:.3f}, R={r:.3f}, mAP50={m50:.3f}, mAP50-95={m95:.3f}, F1={f1:.3f}")

        cm = m.confusion_matrix.matrix if hasattr(m, "confusion_matrix") else None
        if cm is not None:
            tp = cm.diagonal().sum()
            fp = cm.sum(axis=0).sum() - tp
            fn = cm.sum(axis=1).sum() - tp
            print(f"TP={tp}, FP={fp}, FN={fn}")

print("\nSTEP 5: Inference Script")
print("-" * 40)

inference_code = f'''import torch
from ultralytics import YOLO

model = YOLO("{model_path if model_path else 'path/to/model.pt'}")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def detect(image_path):
    results = model.predict(
        source=image_path,
        conf=0.001,
        iou=0.25,
        max_det=3000,
        imgsz=640,
        device=device,
        verbose=False,
        agnostic_nms=True
    )
    return results[0]

def detect_tta(image_path):
    outputs = []
    for size in [640, 704, 768]:
        res = model.predict(
            source=image_path,
            conf=0.001,
            iou=0.3,
            imgsz=size,
            device=device,
            verbose=False
        )
        outputs.append(res[0])
    return max(outputs, key=lambda x: len(x.boxes) if x.boxes else 0)
'''

with open(f"{WORK_DIR}/survue_detect.py", "w") as f:
    f.write(inference_code)

print("Saved: survue_detect.py")

print("\n" + "=" * 80)
print("TRAINING COMPLETE")
print("=" * 80)


SURVUE - TRAINING PIPELINE



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip



STEP 1: Dataset
----------------------------------------
Dataset found
Created data_recall.yaml

STEP 2: Training Settings
----------------------------------------
Device: cuda

STEP 3: Strategy Selection
----------------------------------------

Training strategy_2_large
----------------------------------------
New https://pypi.org/project/ultralytics/8.3.236 available 😃 Update with 'pip install -U ultralytics'
WARNING ⚠️ 'crop_fraction' is deprecated and will be removed in the future.
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in the future.
Ultralytics 8.3.231 🚀 Python-3.10.16 torch-2.5.1 CUDA:0 (Tesla P100-PCIE-12GB, 12194MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=3.0, cache=False, cfg=None, classes=None, close_mosaic=10, cls=2.0, compile=False, conf=0.001, copy_paste=0.4, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/courses/CS5330.202610/students/reddy.ada/runs/detect/val9/runs/sur